In [17]:
import json
import pandas as pd

In [18]:
# Load data from Excel
filepath = "../Data/Mode1/Output/mode_1_release_1_all_output.xlsx"
raw_data = pd.read_excel(filepath)

In [19]:
def process_raw_data(raw_data):
    processed_rows = []
    
    for _, row in raw_data.iterrows():
        group = row["group"]
        task_data = json.loads(row["data"])["Data"]["taskData"][0]  # Parse the JSON data
        
        grouped_data = {}
        for key, value in task_data.items():
            prefix, topic = key.split('_', 1)
            if prefix not in grouped_data:
                grouped_data[prefix] = {}
            grouped_data[prefix][topic] = value

        # Convert grouped data into records for DataFrame
        for prefix, topics in grouped_data.items():
            record = {"group": group, "prefix": prefix, **topics}
            processed_rows.append(record)
    
    return pd.DataFrame(processed_rows)

def extract_true_values(df):
    def find_true_value(mapping):
        # Identify the key with a True value
        for key, value in mapping.items():
            if value:
                return key
        return None

    # Apply to all topic columns
    topic_columns = df.columns.difference(["group", "prefix"])
    for column in topic_columns:
        df[column] = df[column].apply(find_true_value)

    return df

def count_unique_values(df):
    unique_counts = {}
    for column in df.columns.difference(["group", "prefix"]):
        unique_counts[column] = df[column].value_counts().to_dict()
    return unique_counts

In [ ]:
# Process and transform data
processed_data = process_raw_data(raw_data)

# Extract only the "True" values
data_with_true_values = extract_true_values(processed_data)

# Count unique values
unique_counts = count_unique_values(data_with_true_values)

# Display the results
print(data_with_true_values)
print("\nUnique counts:", unique_counts)


      group prefix validness       relatedness appropriateness
0   group_1   3243       Yes  Slightly Related              No
1   group_1   5230       Yes           Related              No
2   group_1   8141       Yes           Related              No
3   group_1  10373       Yes  Slightly Related              No
4   group_1  10555       Yes           Related              No
..      ...    ...       ...               ...             ...
95  group_5  19758       Yes  Strongly Related              No
96  group_5  23129       Yes  Strongly Related              No
97  group_5  24508       Yes  Strongly Related             Yes
98  group_5  24618       Yes  Strongly Related              No
99  group_5  25372       Yes  Strongly Related              No

[100 rows x 5 columns]

Unique counts: {'appropriateness': {'No': 84, 'Yes': 16}, 'relatedness': {'Related': 35, 'Strongly Related': 34, 'Slightly Related': 22, 'Not Related': 9}, 'validness': {'Yes': 100}}
